In [94]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

In [95]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [96]:
tokens = tokenizer.encode('kinda ok', return_tensors='pt')

In [97]:
result = model(tokens)

In [98]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.1173, -0.3126,  1.0229,  0.5143, -0.2423]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [99]:
result.logits

tensor([[-1.1173, -0.3126,  1.0229,  0.5143, -0.2423]],
       grad_fn=<AddmmBackward0>)

In [100]:
int(torch.argmax(result.logits))+1

3

In [104]:
# r = requests.get('https://www.yelp.com/biz/girl-and-the-goat-chicago?osq=Restaurants')
r = requests.get('https://www.rottentomatoes.com/m/the_wolf_of_wall_street_2013/reviews?type=user')
soup = BeautifulSoup(r.text, 'html.parser')
# regex = re.compile('.*comment.*') #Find Comment Class to sprape the comment// class of the content
regex = re.compile('.*audience-reviews__review js-review-text.*') 
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [105]:
results[0]

<p class="audience-reviews__review js-review-text" data-qa="review-text" slot="content">
                      This movie is an absolute masterpiece and one of my all-time favorites. Its gripping storyline, fantastic performances by the cast, and stunning cinematography make it a must-watch for anyone who enjoys great cinema. If you haven't seen it yet, I highly recommend that you do so as soon as possible.
                  </p>

In [106]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [107]:
df['review'].iloc[0]

"\n                      This movie is an absolute masterpiece and one of my all-time favorites. Its gripping storyline, fantastic performances by the cast, and stunning cinematography make it a must-watch for anyone who enjoys great cinema. If you haven't seen it yet, I highly recommend that you do so as soon as possible.\n                  "

In [108]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [109]:
sentiment_score(df['review'].iloc[0])

5

In [110]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [111]:
df

,review,sentiment
0,\n This movie is an absol...,5
1,\n What was the point of ...,1
2,\n The Wolf of Wall Stree...,5
3,\n I really liked this fi...,4
4,\n A classic rise and fal...,5
5,\n What an interesting Mo...,4
6,\n Incredible film that i...,5
7,\n Absolute Must See! Leo...,5
8,\n I studied econ in uni ...,2
9,\n Biggest thing about it...,3


In [112]:
average_sentiment = df['sentiment'].mean()
median_sentiment = df['sentiment'].median()

print(f'Average Sentiment: {average_sentiment}')
print(f'Median Sentiment: {median_sentiment}')

Average Sentiment: 3.4
Median Sentiment: 4.0
